In [ ]:
import pandas as pd
import redis

In [ ]:
# Load the data from Excel file
excel_file_path = r'extracted.xlsx'  # Change this to your actual file path
df = pd.read_excel(excel_file_path)

# Assuming the Redis server is on localhost and the default port, with no password
# Change these to your actual Redis server details

redis_host = "192.168.1.4"
redis_port = 6379
redis_password = ""

In [ ]:
import pandas as pd
import redis

def connect_to_redis(redis_host, redis_port, redis_password):
    try:
        # Connect to Redis server
        r = redis.StrictRedis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)
        return r
    except redis.exceptions.ConnectionError as e:
        print(f"Failed to connect to Redis server: {e}")
        return None

def fetch_near_miss_events(r):
    try:
        # Use the SCAN command to fetch keys
        near_miss_events = []
        cursor = '0'
        while True:
            cursor, keys = r.scan(cursor=cursor, match='event:*', count=100)
            for key in keys:
                event_data = r.hgetall(key)
                if event_data.get('104.Classification') == 'Near-miss' and len(near_miss_events) < 10:
                    near_miss_events.append(event_data)
                    if len(near_miss_events) == 10:
                        break
            if cursor == '0' or len(near_miss_events) == 10:
                break  # Break when scan has returned to the start or we've found 10 events
        return near_miss_events
    except Exception as e:
        print(e)

def events_to_dataframe(near_miss_events):
    # Convert the list of dictionaries to a pandas DataFrame
    events_df = pd.DataFrame(near_miss_events)
    return events_df

# Fetch near miss events from Redis and store it in a DataFrame
r = connect_to_redis(redis_host, redis_port, redis_password)
if r is not None:
    near_miss_events = fetch_near_miss_events(r)
    events_df = events_to_dataframe(near_miss_events)

    # Print the resulting DataFrame
    print(events_df)
